<a href="https://colab.research.google.com/github/MHA535/Marketing_Automations_Notebooks_With_GPT/blob/main/Automatic_Newsjacking_Ideation_and_Trend_Analysis_(Public).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install google-search-results
!pip install newspaper3k
!pip install sentence-transformers
!pip install sklearn


In [ ]:
import openai
import pandas as pd
import concurrent.futures
import os
import time

# Load API key from environment variable (ensure you've set this in your environment)
from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')# Ensure your API key is set in your environment

# Define the GPT model name
MODEL_NAME = "gpt-4"

def generate_text(prompt):
    try:
        response = openai.Completion.create(
            model=MODEL_NAME,
            prompt=prompt,
            max_tokens=1500,
            temperature=0.6,
            n=1,
        )
        return response.choices[0].text.strip()
    except openai.OpenAIError as e:
        print(f"OpenAI API error: {e}")
        return ""  # Return an empty string if an error occurs
    except Exception as e:
        print(f"Unexpected error: {e}")
        return ""

def generate_subtopics(topic):
    prompt = f"Generate 40 common subtopics related to {topic}"
    return generate_text(prompt).split("\n")

def generate_subsubtopics(subtopic):
    prompt = f"Generate 20 common subsubtopics related to {subtopic}"
    return generate_text(prompt).split("\n")

def generate_questions(topic, subtopic, subsubtopic):
    prompt = f"""You are an all-knowing AI that specializes in search behavior, SEO, and search intent.
    You are especially good at predicting questions related to topics.
    Please generate 30 of the most common questions related to {subsubtopic}
    within the context of {subtopic} in the larger topic of {topic}."""

    questions = generate_text(prompt).split("\n")
    return [question.split(". ")[-1] for question in questions if question.strip()]

def process_subsubtopic(topic, subtopic, subsubtopic):
    questions = generate_questions(topic, subtopic, subsubtopic)
    data = {
        "topic": topic,
        "subtopic": subtopic,
        "subsubtopic": subsubtopic,
    }
    for i, question in enumerate(questions):
        data[f"Question {i + 1}"] = question
    return data

def main(topic):
    # Generate subtopics
    subtopics = generate_subtopics(topic)
    data = []

    # Use ThreadPoolExecutor for parallel processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:  # Limiting to 5 concurrent threads
        futures = []

        # Generate subsubtopics and process questions in parallel
        for subtopic in subtopics:
            subsubtopics = generate_subsubtopics(subtopic)
            for subsubtopic in subsubtopics:
                futures.append(executor.submit(process_subsubtopic, topic, subtopic, subsubtopic))

        for future in concurrent.futures.as_completed(futures):
            data.append(future.result())

    # Convert the collected data to a DataFrame
    df = pd.DataFrame(data)
    return df

if __name__ == "__main__":
    topic = "Rehan Centers"
    df = main(topic)
    print(df)


In [ ]:
content_df.to_csv('text.csv')